# NBA Most Valorated Player Analysis

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from time import sleep
import requests
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

- votaciones: https://www.basketball-reference.com/awards/awards_1989.html#all_nba_mvp